In [64]:
import numpy as np
import pandas as pd
import pickle as pk
from imblearn.under_sampling import RandomUnderSampler
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, MinMaxScaler
from tensorflow import keras
import tensorflow as tf
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasClassifier
from sklearn.metrics import classification_report

In [65]:
def load_data():
    samplers = [RandomUnderSampler(sampling_strategy='majority'), RandomOverSampler(sampling_strategy='all')]
    df = pd.read_csv('./train_combined_Species.csv')
    df.drop(df.columns[0], axis=1, inplace=True)
    df = df.fillna(0).drop_duplicates()

    pd.options.mode.chained_assignment = None  # default='warn'
    df0 = df.loc[(df['label'] == 0)]
    df1 = df.loc[(df['label'] == 1)]
    df2 = df.loc[(df['label'] == 2)]
    df3 = df.loc[(df['label'] == 3)]
    df4 = df.loc[(df['label'] == 4)]
    df_one = df3
    df_rest = pd.concat([df0, df1, df2, df4], axis=0)
    df_rest['label'].replace([0,1,2,3,4], 0, inplace=True)
    df_one['label'].replace(3, 1, inplace=True) # Value needs change
    df = pd.concat([df_rest, df_one], axis=0)


    weight_for_0 = (1 / df_rest.shape[0]) * (df.shape[0] / 2.0)
    weight_for_1 = (1 / df_one.shape[0]) * (df.shape[0] / 2.0)

    #column_cut_off = int(90/100*len(df))
    #b = (df == 0).sum(axis='rows')
    #df = df[ b[ b <= column_cut_off].index.values ]

    inputvalues = df.drop(['label'], axis=1)
    outputvalues = df['label']
    X, y = inputvalues.values, outputvalues.values
    X = X.astype('float32')

    #scaler = MinMaxScaler().fit(X)
    #pk.dump(scaler, open("scaler.pkl","wb"))
    #X = scaler.transform(X)

    #pca = PCA(n_components=400).fit(X_scaled)
    #pk.dump(pca, open("pca.pkl","wb"))
    #X = pca.transform(X)

    #y = y.reshape(-1, 1)
    #oneHotEncoder = OneHotEncoder(categories=[['0', '1', '2', '3', '4']], sparse=False).fit(y)
    #pk.dump(oneHotEncoder, open("oneHotEncoder.pkl","wb"))
    #y = oneHotEncoder.transform(y)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1234)
    #X_train, y_train = samplers[0].fit_resample(X_train, y_train)


    n_features = X_train.shape[1]
    return df, X_train, X_test, y_train, y_test, n_features, weight_for_0, weight_for_1

In [74]:
def create_model(activation, dropout_rate, init_mode, kernel_regularizer_val1, kernel_regularizer_val2,
                 bias_regularizer_val, activity_regularizer_val, label_count, n_features, opt_type,
                 learning_rate, dropout, batchN, size1, size2, size3):
    model = Sequential()
    if opt_type == 1:
        opt = keras.optimizers.Adam(
            learning_rate=learning_rate)
    else:
        opt = tf.keras.optimizers.experimental.SGD(
            learning_rate=learning_rate)

    sizes = [size1, size1, size2, size2, size3, size3]
    inputlayer = True
    i = -1
    while i < 2:
        i = i + 1
        create_layer(model=model, size=sizes[i], activation=activation, dropout=dropout,
                     dropout_rate=dropout_rate, batchN=batchN, inputlayer=inputlayer, init_mode=init_mode,
                     kernel_regularizer_val1=kernel_regularizer_val1, kernel_regularizer_val2=kernel_regularizer_val2,
                     bias_regularizer_val=bias_regularizer_val, activity_regularizer_val=activity_regularizer_val,
                     n_features=n_features)
        inputlayer = False
    model.add(Dense(label_count, activation="sigmoid"))
    METRICS = [
        keras.metrics.TruePositives(name='tp'),
        keras.metrics.FalsePositives(name='fp'),
        keras.metrics.TrueNegatives(name='tn'),
        keras.metrics.FalseNegatives(name='fn'),
        keras.metrics.BinaryAccuracy(name='accuracy'),
        keras.metrics.Precision(name='precision'),
        keras.metrics.Recall(name='recall'),
        keras.metrics.AUC(name='auc'),
        keras.metrics.AUC(name='prc', curve='PR'), ]

    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=METRICS)
    return model


def create_layer(model, size, activation, dropout, dropout_rate, batchN, inputlayer, init_mode,
                 kernel_regularizer_val1, kernel_regularizer_val2, bias_regularizer_val, activity_regularizer_val,
                 n_features):
    if inputlayer:
        model.add(Dense(size, activation=activation, kernel_initializer=init_mode,
                        kernel_regularizer=tf.keras.regularizers.L1L2(l1=kernel_regularizer_val1,
                                                                      l2=kernel_regularizer_val2),
                        bias_regularizer=tf.keras.regularizers.L2(bias_regularizer_val),
                        activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_val),
                        input_shape=(n_features,)))
    else:
        model.add(Dense(size, activation=activation, kernel_initializer=init_mode,
                        kernel_regularizer=tf.keras.regularizers.L1L2(l1=kernel_regularizer_val1,
                                                                      l2=kernel_regularizer_val2),
                        bias_regularizer=tf.keras.regularizers.L2(bias_regularizer_val),
                        activity_regularizer=tf.keras.regularizers.L2(activity_regularizer_val)))
    if dropout:
        model.add(tf.keras.layers.Dropout(rate=dropout_rate))
    if batchN:
        model.add(tf.keras.layers.BatchNormalization())

In [67]:
# search for optimal hyperparameter
def hyper_grid_search():
    # creating the Keras Model
    model = KerasClassifier(model=create_model, verbose=1)

    # Load the dataset
    df, X_train, X_test, y_train, y_test, n_features = load_data()

    # Hyperparameter for the Model
    label_count = [5]
    n_features = [n_features]
    batch_sizes = [64]
    epochss = [60, 90] # 74
    activations = ['relu'] # activations = ['softmax', 'softplus', 'softsign', 'relu', 'tanh', 'sigmoid', 'hard_sigmoid', 'linear']
    init_mode = ['glorot_normal'] # init_mode = ['uniform', 'lecun_uniform', 'normal', 'zero', 'glorot_normal', 'glorot_uniform', 'he_normal', 'he_uniform']
    dropout_rates = [0.1, 0.2]
    kernel_regularizer_val1s = [1e-6, 1e-5] #[1e-6, 1e-5, 1e-4, 1e-3]
    kernel_regularizer_val2s = [1e-5, 1e-4] #[1e-5, 1e-4, 1e-3, 1e-2]
    bias_regularizer_vals = [1e-5, 1e-4] #[1e-5, 1e-4, 1e-3, 1e-2]
    activity_regularizer_vals = [1e-6, 1e-5] #[1e-6, 1e-5, 1e-4, 1e-3]
    opt_types = [1]
    learning_rates = [1e-05, 1e-06, 1e-07]  # [0.0000001, 0.000001, 0.00001, 0.0001]
    dropouts = [True]
    batchNs = [True]
    sizes1 = [64]  # [4, 8, 16, 32]
    sizes2 = [32]
    sizes3 = [4]
    param_grid = dict(model__activation=activations, model__dropout_rate=dropout_rates, model__init_mode=init_mode,
                      model__kernel_regularizer_val1=kernel_regularizer_val1s,
                      model__kernel_regularizer_val2=kernel_regularizer_val2s,
                      model__bias_regularizer_val=bias_regularizer_vals,
                      model__activity_regularizer_val=activity_regularizer_vals,
                      model__label_count=label_count, model__n_features=n_features,
                      model__opt_type=opt_types, model__learning_rate=learning_rates, model__dropout=dropouts,
                      model__batchN=batchNs,
                      model__size1=sizes1, model__size2=sizes2, model__size3=sizes3, batch_size=batch_sizes,
                      epochs=epochss)

    grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=3)
    grid_result = grid.fit(X_train, y_train, validation_data=(X_test, y_test), verbose=0)

    # summarize results
    print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
    means = grid_result.cv_results_['mean_test_score']
    stds = grid_result.cv_results_['std_test_score']
    params = grid_result.cv_results_['params']
    for mean, stdev, param in zip(means, stds, params):
        print("%f (%f) with: %r" % (mean, stdev, param))

In [81]:
from matplotlib import pyplot as plt


def train(iters):
    df, X_train, X_test, y_train, y_test, n_features, weight_for_0, weight_for_1 = load_data()
    i = iters


    class_weight = {0: weight_for_0, 1: weight_for_1}

    while i > 0:
        i = i - 1
        model = create_model(activation='relu', dropout_rate=0.1, init_mode='he_normal', kernel_regularizer_val1=1e-6,
                             kernel_regularizer_val2=1e-5,
                             bias_regularizer_val=1e-5, activity_regularizer_val=1e-6, label_count=1,
                             n_features=n_features,
                             opt_type=1, learning_rate=1e-06, dropout=True, batchN=False, size1=12, size2=8, size3=4)

        # early_stopping = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)
        callbacks_list = []
        result = model.fit(X_train, y_train, epochs=75, batch_size=128, verbose=2, class_weight=class_weight, callbacks=callbacks_list)

        eval_model(model, X_test, y_test)


def eval_model(model, X_test, y_test):
    #y_test = np.argmax(y_test, axis=1)  # Convert one-hot to index
    y_pred = np.argmax(model.predict(X_test), axis=-1)
    print(classification_report(y_test, y_pred))
    print(y_pred)
    model.save('Species_FHHED.h5')





In [82]:
train(1)
# hyper_grid_search()

# model.save_weights('./' + "3x12294_500_279_0.5")

# model = create_model(label_count)
# model.load_weights('./' + "3x12294_500_279_0.5")

Epoch 1/75
7/7 - 2s - loss: 0.1453 - tp: 1.0000 - fp: 8.0000 - tn: 774.0000 - fn: 21.0000 - accuracy: 0.9639 - precision: 0.1111 - recall: 0.0455 - auc: 0.5123 - prc: 0.0393 - 2s/epoch - 246ms/step
Epoch 2/75
7/7 - 0s - loss: 0.1437 - tp: 1.0000 - fp: 10.0000 - tn: 772.0000 - fn: 21.0000 - accuracy: 0.9614 - precision: 0.0909 - recall: 0.0455 - auc: 0.5122 - prc: 0.0344 - 64ms/epoch - 9ms/step
Epoch 3/75
7/7 - 0s - loss: 0.1471 - tp: 0.0000e+00 - fp: 6.0000 - tn: 776.0000 - fn: 22.0000 - accuracy: 0.9652 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.4917 - prc: 0.0259 - 70ms/epoch - 10ms/step
Epoch 4/75
7/7 - 0s - loss: 0.1423 - tp: 1.0000 - fp: 12.0000 - tn: 770.0000 - fn: 21.0000 - accuracy: 0.9590 - precision: 0.0769 - recall: 0.0455 - auc: 0.5082 - prc: 0.0297 - 75ms/epoch - 11ms/step
Epoch 5/75
7/7 - 0s - loss: 0.1452 - tp: 0.0000e+00 - fp: 11.0000 - tn: 771.0000 - fn: 22.0000 - accuracy: 0.9590 - precision: 0.0000e+00 - recall: 0.0000e+00 - auc: 0.5117 - prc: 0.0299 - 77m

C:\Users\Flo13\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Flo13\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Flo13\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,